# AgentAction and AgentFinish

In [1]:
%load_ext dotenv
%dotenv

In [2]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain_community.vectorstores import Chroma

from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI

from langchain.tools.retriever import create_retriever_tool
from langchain.tools import tool

from langchain import hub

from platform import python_version

from langchain.agents import create_tool_calling_agent, AgentExecutor

In [3]:
wikipedia_tool = WikipediaQueryRun(api_wrapper = WikipediaAPIWrapper())

In [4]:
vectorstore = Chroma(persist_directory = "./intro-to-ds-lectures", 
                     embedding_function = OpenAIEmbeddings())

retriever = vectorstore.as_retriever(search_type = 'mmr', 
                                     search_kwargs = {'k':3, 
                                                      'lambda_mult':0.7})

retriever_tool = create_retriever_tool(retriever = retriever, 
                                       name = "Introduction-to-Data-and-Data-Science-Course-Lectures", 
                                       description = '''For any questions regarding 
                                       the Introduction to Data and Data Science course, you must use this tool.''')

In [5]:
@tool
def get_python_version() -> str:
    ''' Useful for questions regarding the version of Python currently used. '''
    return python_version()

In [6]:
tools = [wikipedia_tool, retriever_tool, get_python_version]

In [7]:
chat = ChatOpenAI(model_name = 'gpt-4-0125-preview', 
                  temperature = 0,
                  model_kwargs = {'seed':365})

In [8]:
chat_prompt_template = hub.pull("hwchase17/openai-tools-agent")

In [9]:
agent = create_tool_calling_agent(llm = chat, 
                                  tools = tools, 
                                  prompt = chat_prompt_template)

In [14]:
agent.invoke({"input":"Could you tell me the version of Python I'm currently using?", 
              "intermediate_steps":response["intermediate_steps"]})

AgentFinish(return_values={'output': 'The version of Python currently being used is 3.10.10.'}, log='The version of Python currently being used is 3.10.10.')

In [10]:
agent_executor = AgentExecutor(agent = agent, 
                               tools = tools, 
                               verbose = True, 
                               return_intermediate_steps = True)

In [11]:
response = agent_executor.invoke({"input":"Could you tell me the version of Python I'm currently using?"})



> Entering new AgentExecutor chain...

Invoking: `get_python_version` with `{}`


3.10.10The version of Python currently being used is 3.10.10.

> Finished chain.


In [12]:
response

{'input': "Could you tell me the version of Python I'm currently using?",
 'output': 'The version of Python currently being used is 3.10.10.',
 'intermediate_steps': [(ToolAgentAction(tool='get_python_version', tool_input={}, log='\nInvoking: `get_python_version` with `{}`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_lo6ih5XF62vPNrCxcMkKZIqh', 'function': {'arguments': '{}', 'name': 'get_python_version'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-9464a78d-3eb1-4b38-a18e-53537d2dc453', tool_calls=[{'name': 'get_python_version', 'args': {}, 'id': 'call_lo6ih5XF62vPNrCxcMkKZIqh'}], tool_call_chunks=[{'name': 'get_python_version', 'args': '{}', 'id': 'call_lo6ih5XF62vPNrCxcMkKZIqh', 'index': 0}])], tool_call_id='call_lo6ih5XF62vPNrCxcMkKZIqh'),
   '3.10.10')]}